In [198]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModelForSequenceClassification
import torch

In [7]:
import pandas as pd

intermediate_outputs = pd.read_json('/data/hyeryung/mucoco/outputs/toxicity/llm/r7kykwge/outputs_epsilon0.9.txt.intermediate', lines=True)
intermediate_outputs = intermediate_outputs.explode('generations').reset_index(drop=True)

In [14]:
intermediate_outputs_eda = intermediate_outputs.loc[intermediate_outputs['generations'].apply(len) != 0].reset_index(drop=True)

In [153]:
intermediate_outputs_eda['prompt'] = intermediate_outputs_eda['prompt'].apply(lambda x: x['text'])

In [23]:
intermediate_outputs_eda['masked_sentences'] = intermediate_outputs_eda['generations'].apply(lambda x: [item[1] for item in x.items() if 'mask' in item[0]])

In [162]:
intermediate_outputs_eda['masked_sentences_joined_by_prompt'] = intermediate_outputs_eda[['prompt','masked_sentences']].apply(lambda x: [' ' .join([x['prompt'], y]) for y in x['masked_sentences']], axis=1)

In [163]:
intermediate_outputs_eda['masked_sentences_joined_by_prompt'][0][0]

'A convicted German criminal known as the “broomstick robber” has been charged with blackmail for allegedly holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask><mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask><mask> activities.<mask> "broomstick<mask><mask> saga captivated the nation and left many stunned by the audacity of his crimes.'

In [164]:
intermediate_outputs_eda['masked_sentences'][0][0]

'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask><mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask><mask> activities.<mask> "broomstick<mask><mask> saga captivated the nation and left many stunned by the audacity of his crimes.'

In [165]:
all_masked_sentences = intermediate_outputs_eda['masked_sentences'].agg(sum)

In [166]:
test_sent = all_masked_sentences[0]

In [167]:
test_sent

'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask><mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask><mask> activities.<mask> "broomstick<mask><mask> saga captivated the nation and left many stunned by the audacity of his crimes.'

In [74]:
import re
from collections import defaultdict


def analyze_span_lengths_and_count(text):
    mask_matches = list(re.finditer('<mask>', text))

    mask_info_dict= defaultdict(list)
    prev_mask = None
    span_count = 0
    curr_span_length = 1
    for i, mask in enumerate(mask_matches):
        
        if i == 0:
            mask_info_dict[span_count].append(i)
            
        else:
            if prev_mask.span()[1] == mask.span()[0]:
                mask_info_dict[span_count].append(i)
                curr_span_length += 1
            else:
                span_count += 1
                mask_info_dict[span_count].append(i)
                curr_span_length = 1
        prev_mask = mask

    span_lengths = []
    for span_id, span_len in mask_info_dict.items():
        
        span_lengths.append(len(span_len))
    return mask_info_dict, span_lengths

In [75]:
mask_info_dicts = []
span_lengths_es = []
for test_sent in all_masked_sentences:
    
    mask_info_dict, span_lengths = analyze_span_lengths_and_count(test_sent)
    mask_info_dicts.append(mask_info_dict)
    span_lengths_es.append(span_lengths)

In [86]:
mlm = AutoModelForMaskedLM.from_pretrained('roberta-large')
mlm_tokenizer = AutoTokenizer.from_pretrained('roberta-large')

In [168]:
test_sent

'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask><mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask><mask> activities.<mask> "broomstick<mask><mask> saga captivated the nation and left many stunned by the audacity of his crimes.'

In [169]:
## merge masks
test_sent_merged = re.sub(r"(<mask>)+", "<mask>", test_sent)


In [170]:
test_sent_tokens = mlm_tokenizer(test_sent_merged,return_tensors="pt",add_special_tokens=False).input_ids

In [171]:
locations = torch.where(torch.Tensor(test_sent_tokens) == mlm_tokenizer.mask_token_id) 

In [151]:
test_sent_short = ' '.join(test_sent.split()[:20])
## merge masks
test_sent_short_merged = re.sub(r"(<mask>)+", "<mask>", test_sent_short)

In [152]:
test_sent_short_merged

"<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge"

In [208]:
grammar_model = AutoModelForSequenceClassification.from_pretrained('textattack/roberta-base-CoLA')

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [213]:
test_sent_tokens

tensor([[25176,    62,    10,  6755,     9, 21356,   634,    10, 39546, 19650,
         31890,    25,    10,  1751,     4,    20,   313,  2288, 50264,   813,
             8,   916,   150,  5783,   418,     8,  7398,   257,  6058,     4,
           572,    10,   251,   803,     6,     5,   249,  2312,     7,  1349,
           123,   159,     8,   836,  1103,   136,   123,    13,    39, 50264,
          1713,     4, 50264,    22,   428,  4294, 19650, 50264, 17466, 13363,
         23907,     5,  1226,     8,   314,   171, 12144,    30,     5,  9818,
         18583,     9,    39,  3474,     4]])

In [217]:
span_lengths[cursor_idx]

2

In [219]:
span_lengths = span_lengths_es[0]
cursor_idx = 0
cursor = locations[1][cursor_idx]
cursor_offset = 0

comprehensive_hypotheses = []
while True:
    with torch.no_grad():
        comprehensive_hypotheses.append(torch.cat((test_sent_tokens[:, :cursor], test_sent_tokens[:, cursor+1:]), dim=-1))
        
        predictions = mlm(test_sent_tokens).logits
        predictions[:, :, [0,1,2,3,50264]] = -10000
        top3 = torch.topk(predictions[:, cursor,:], k=3, dim=-1).indices
        comprehensive_hypotheses.append(torch.cat((torch.tile(test_sent_tokens[:, :cursor], dims= (3,1)), top3.reshape(-1,1), torch.tile(test_sent_tokens[:, cursor+1:], dims= (3,1))), dim=-1))
        hypotheses = torch.cat((torch.tile(test_sent_tokens[:, :cursor], dims= (3,1)), top3.reshape(-1,1), torch.tile(test_sent_tokens[:, cursor:], dims= (3,1))), dim=-1)
        cursor_offset += 1
        
        iteration_count = max(span_lengths[cursor_idx], 5)
        
        for _ in range(iteration_count):
            hypotheses_texts = mlm_tokenizer.batch_decode(hypotheses)
            predictions = mlm(hypotheses).logits
            top3 = torch.topk(predictions[:, cursor + cursor_offset], k=3, dim=-1).indices
            print(torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset+1:], dims= (3,1))), dim=-1))
            hypotheses_scores = torch.softmax(grammar_model(torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset+1:], dims= (3,1))), dim=-1)).logits,dim=-1)
            # print(hypotheses_scores[: ,1])
            # break
            
            
            comprehensive_hypotheses.append(torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset+1:], dims= (3,1))), dim=-1))
            hypotheses = torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset:], dims= (3,1))), dim=-1)
            cursor_offset += 1
            if (hypotheses[:, cursor+cursor_offset-1] == hypotheses[:, cursor+cursor_offset]).sum() > 0:
                hypotheses = hypotheses[torch.randint(0, len(hypotheses), (10,)), :]
                break
            else:
                hypotheses = hypotheses[torch.randint(0, len(hypotheses), (10,)), :]
            
        
        
        break
        
        test_sent_tokens()
        test_sent_tokens

tensor([[25176,    62,    10,  6755,     9, 21356,   634,    10, 39546, 19650,
         31890,    25,    10,  1751,     4,    20,   313,  2288,  3711,     5,
           813,     8,   916,   150,  5783,   418,     8,  7398,   257,  6058,
             4,   572,    10,   251,   803,     6,     5,   249,  2312,     7,
          1349,   123,   159,     8,   836,  1103,   136,   123,    13,    39,
         50264,  1713,     4, 50264,    22,   428,  4294, 19650, 50264, 17466,
         13363, 23907,     5,  1226,     8,   314,   171, 12144,    30,     5,
          9818, 18583,     9,    39,  3474,     4],
        [25176,    62,    10,  6755,     9, 21356,   634,    10, 39546, 19650,
         31890,    25,    10,  1751,     4,    20,   313,  2288,  4487,   258,
           813,     8,   916,   150,  5783,   418,     8,  7398,   257,  6058,
             4,   572,    10,   251,   803,     6,     5,   249,  2312,     7,
          1349,   123,   159,     8,   836,  1103,   136,   123,    13,    39,


In [220]:
[mlm_tokenizer.batch_decode(x) for x in comprehensive_hypotheses]

[['holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.'],
 ['holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly threatened staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.',
  'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly attacked staff and customers while demanding money and valuables. After a long investigation, the police managed to track him dow

In [178]:
mlm_tokenizer.batch_decode(hypotheses_2)

['holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly threatened the<mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.',
 'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly attacked both<mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.',
 'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly assaulted store<mask> staff and customers while demanding money and valuables. After a long investigati

In [112]:
test_sent

"<mask><mask> know can actually hurt us.<mask><mask><mask> can lead to<mask><mask><mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don't know are exactly what we need to discover to grow and make better choices in the future."

In [111]:
mlm_tokenizer.batch_decode(hypotheses_2)

["<s> alreadyWe<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don't know are exactly what we need to discover to grow and make better choices in the future.</s>",
 "<s> doKnowing<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don't know are exactly what we need to discover to grow and make better choices in the future.</s>",
 "<s> shouldNot<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don

In [56]:
mask_info_dict

defaultdict(list,
            {0: [<re.Match object; span=(93, 99), match='<mask>'>,
              <re.Match object; span=(99, 105), match='<mask>'>],
             1: [<re.Match object; span=(265, 271), match='<mask>'>,
              <re.Match object; span=(271, 277), match='<mask>'>],
             2: [<re.Match object; span=(289, 295), match='<mask>'>],
             3: [<re.Match object; span=(307, 313), match='<mask>'>,
              <re.Match object; span=(313, 319), match='<mask>'>]})

In [44]:
mask.span()

(313, 319)

In [45]:
mask_info_dict

defaultdict(list,
            {0: [<re.Match object; span=(93, 99), match='<mask>'>,
              <re.Match object; span=(99, 105), match='<mask>'>,
              <re.Match object; span=(265, 271), match='<mask>'>,
              <re.Match object; span=(271, 277), match='<mask>'>,
              <re.Match object; span=(289, 295), match='<mask>'>,
              <re.Match object; span=(307, 313), match='<mask>'>,
              <re.Match object; span=(313, 319), match='<mask>'>]})